In [1]:
# Prepare the environment
try:
    from easypip import easyimport
except ModuleNotFoundError:
    from subprocess import run

    assert (
        run(["pip", "install", "easypip"]).returncode == 0
    ), "Could not install easypip"
    from easypip import easyimport

easyimport("swig")
easyimport("bbrl_utils").setup(maze_mdp=True)

import os
import copy
import bbrl_gymnasium  # noqa: F401
import torch
import torch.nn as nn
from bbrl.agents import Agent, Agents, TemporalAgent
from bbrl_utils.algorithms import EpochBasedAlgo
from bbrl_utils.nn import build_mlp, setup_optimizer, soft_update_params
from bbrl_utils.notebook import setup_tensorboard
from bbrl.visu.plot_policies import plot_policy
from omegaconf import OmegaConf
from pettingzoo.butterfly import knights_archers_zombies_v10
from torch.distributions import Normal


[easypip] Installing bbrl_utils
/Users/angeleramauge/anaconda3/lib/python3.11/site-packages/bbrl_utils/notebook.py:46: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # noqa: F401


In [2]:
class DiscreteQAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        self.model = build_mlp(
            [state_dim] + list(hidden_layers) + [action_dim], activation=nn.ReLU()
        )

    def forward(self, t, **kwargs):
        obs = self.get(("env/env_obs", t))
        q_values = self.model(obs)
        self.set((f"{self.prefix}q_values", t), q_values)

class ArgmaxActionSelector(Agent):
    """BBRL agent that selects the best action based on Q(s,a)"""

    def forward(self, t: int, **kwargs):
        q_values = self.get(("q_values", t))
        action = q_values.argmax(-1)
        self.set(("action", t), action)

class EGreedyActionSelector(Agent):
    def __init__(self, epsilon):
        super().__init__()
        self.epsilon = epsilon

    def forward(self, t: int, **kwargs):
        # Retrieves the q values
        # (matrix nb. of episodes x nb. of actions)
        q_values: torch.Tensor = self.get(("q_values", t))
        size, nb_actions = q_values.shape

        # Flag
        is_random = torch.rand(size) > self.epsilon
        
        # Actions (random / argmax)
        random_action = torch.randint(nb_actions, size=(size,))
        max_action = q_values.argmax(-1)

        # Choose the action based on the is_random flag
        action = torch.where(is_random, random_action, max_action)

        # Sets the action at time t
        self.set(("action", t), action)

class DQN(EpochBasedAlgo):
    def __init__(self, cfg):
        super().__init__(cfg)

        obs_size, act_size = self.train_env.get_obs_and_actions_sizes()

        # Get the two agents (critic and target critic)
        critic = DiscreteQAgent(obs_size, cfg.algorithm.architecture.hidden_size, act_size)
        target_critic = copy.deepcopy(critic).with_prefix("target/")

        # Builds the train agent that will produce transitions
        explorer = EGreedyActionSelector(cfg.algorithm.epsilon)
        self.train_policy = Agents(critic, explorer)

        self.eval_policy = Agents(critic, ArgmaxActionSelector())

        # Creates two temporal agents just for "replaying" some parts
        # of the transition buffer
        self.t_q_agent = TemporalAgent(critic)
        self.t_target_q_agent = TemporalAgent(target_critic)

        # Get an agent that is executed on a complete workspace
        self.optimizer = setup_optimizer(cfg.optimizer, self.t_q_agent)

        self.last_critic_update_step = 0

def dqn_compute_critic_loss(
    cfg, reward, must_bootstrap, q_values, target_q_values, action
):
    """Compute the critic loss

    :param reward: The reward $r_t$ (shape 2 x B)
    :param must_bootstrap: The must bootstrap flag at $t+1$ (shape 2 x B)
    :param q_values: The Q-values (shape 2 x B x A)
    :param target_q_values: The target Q-values (shape 2 x B x A)
    :param action: The chosen actions (shape 2 x B)
    :return: _description_
    """

    # Implement the DQN loss

    # Adapt from the previous notebook and adapt to our case (target Q network)

    #current_q_values = q_values[0] #Q(st,at)
    #next_q_values = q_values[1] #Q(st1,a)

    #max_next_q_values = next_q_values.max(dim=1).values.detach()  #maxQ(st1,a)
    #target = reward[1:] + cfg.algorithm.discount_factor * max_next_q_values * must_bootstrap[1:] #r(st,at) + y* maxQ(st1,a)
    #q_values = torch.gather(current_q_values, dim=1, index=action[0].unsqueeze(-1)).squeeze(-1)  #calcul new Qvals
    
    # Compute critic loss (no need to use must_bootstrap here since we are dealing with "full" transitions)
   # mse = nn.MSELoss()
    #critic_loss = mse(target, q_values)

    
    max_q = target_q_values[1].max(-1).values.detach()
    target = (reward[1] + cfg.algorithm.discount_factor * max_q * must_bootstrap[1].int())
    act = action[0].unsqueeze(-1)
    qvals = torch.gather(q_values[0],dim = 1, index=act).squeeze(-1)

    mse = nn.MSELoss()
    critic_loss = mse(target,qvals)

    return critic_loss

def run(dqn: DQN, compute_critic_loss):
    for rb in dqn.iter_replay_buffers():
        for _ in range(dqn.cfg.algorithm.n_updates):
            rb_workspace = rb.get_shuffled(dqn.cfg.algorithm.batch_size)

            # The q agent needs to be executed on the rb_workspace workspace
            dqn.t_q_agent(rb_workspace, t=0, n_steps=2, choose_action=False)
            with torch.no_grad():
                dqn.t_target_q_agent(rb_workspace, t=0, n_steps=2, stochastic=True)

            q_values, terminated, reward, action, target_q_values = rb_workspace[
                "q_values", "env/terminated", "env/reward", "action", "target/q_values"
            ]

            # Determines whether values of the critic should be propagated
            must_bootstrap = ~terminated

            # Compute critic loss
            critic_loss = compute_critic_loss(
                dqn.cfg, reward, must_bootstrap, q_values, target_q_values, action
            )
            # Store the loss for tensorboard display
            dqn.logger.add_log("critic_loss", critic_loss, dqn.nb_steps)
            dqn.logger.add_log("q_values/min", q_values.max(-1).values.min(), dqn.nb_steps)
            dqn.logger.add_log("q_values/max", q_values.max(-1).values.max(), dqn.nb_steps)
            dqn.logger.add_log("q_values/mean", q_values.max(-1).values.mean(), dqn.nb_steps)

            dqn.optimizer.zero_grad()
            critic_loss.backward()
            torch.nn.utils.clip_grad_norm_(
                dqn.t_q_agent.parameters(), dqn.cfg.algorithm.max_grad_norm
            )
            dqn.optimizer.step()

            # Update target
            if (
                dqn.nb_steps - dqn.last_critic_update_step
                > dqn.cfg.algorithm.target_critic_update
            ):
                dqn.last_critic_update_step = dqn.nb_steps
                copy_parameters(dqn.t_q_agent, dqn.t_target_q_agent)

            dqn.evaluate()

In [3]:
setup_tensorboard("./outputs/tblogs")

Reusing TensorBoard on port 6006 (pid 1953), started 0:19:33 ago. (Use '!kill 1953' to kill it.)

In [33]:
import torch
import torch.nn as nn
import copy
import numpy as np
from pettingzoo.butterfly import knights_archers_zombies_v10
from torch.optim import Adam


class MultiAgentDQN:
    def __init__(self, env, params):
        self.env = env
        self.params = params
        self.agents = env.possible_agents  # List of agents
        obs_sample = env.observation_space(self.agents[0]).sample()
        self.state_dim = np.prod(obs_sample.shape)

        self.action_dim = env.action_space(self.agents[0]).n
        
        # DQN architecture for each agent

        self.critics = {agent: DiscreteQAgent(self.state_dim, params['algorithm']['architecture']['hidden_size'], self.action_dim) for agent in self.agents}
        self.target_critics = {agent: copy.deepcopy(self.critics[agent]) for agent in self.agents}
        
        # Epsilon-greedy explorers
        self.explorers = {agent: EGreedyActionSelector(params['algorithm']['epsilon']) for agent in self.agents}
        
        # Optimizers
        self.optimizers = {agent: Adam(self.critics[agent].parameters(), lr=params['optimizer']['lr']) for agent in self.agents}
        
        self.buffer = []  # A shared buffer for all agents

    def choose_action(self, agent, observation, epsilon=0.02):
        # Choose action based on ε-greedy
        observation_flat = torch.flatten(torch.Tensor(observation))

        q_values = self.critics[agent].model(torch.Tensor(observation_flat).unsqueeze(0))
        if np.random.rand() < epsilon:
            return self.env.action_space(agent).sample()  # Random action
        else:
            return q_values.argmax().item()  # Best action

    def train(self, n_episodes=1000):
        for episode in range(n_episodes):
            self.env.reset(seed=42)  # Reset environment
            for agent in self.env.agent_iter():
                observation, reward, termination, truncation, info = self.env.last()

                if termination or truncation:
                    self.env.step(None)  # No action if agent is done
                else:
                    # Choose action based on ε-greedy DQN policy
                    action = self.choose_action(agent, observation, epsilon=self.params['algorithm']['epsilon'])
                    self.env.step(action)

                    # Collect transitions (s, a, r, s') for training
                    next_obs, next_rew, next_done, _, _ = self.env.last()
                    self.buffer.append((observation, action, reward, next_obs, next_done))

                    # Update agent
                    if len(self.buffer) >= self.params['algorithm']['batch_size']:
                        self.update_agent(agent)

            # Periodically update target networks
            if episode % self.params['algorithm']['target_critic_update'] == 0:
                self.update_target_networks()

    def update_agent(self, agent):
        if len(self.buffer) < self.params['algorithm']['batch_size']:
            return
              
        print(len(self.buffer), self.params['algorithm']['batch_size'])
        print(self.buffer[0])

        batch = np.random.choice(self.buffer, self.params['algorithm']['batch_size'])
        obs, actions, rewards, next_obs, dones = zip(*batch)

        obs = torch.Tensor(obs)
        actions = torch.LongTensor(actions).unsqueeze(-1)
        rewards = torch.Tensor(rewards)
        next_obs = torch.Tensor(next_obs)
        dones = torch.Tensor(dones)

        q_values = self.critics[agent].model(obs).gather(1, actions).squeeze(-1)
        next_q_values = self.target_critics[agent].model(next_obs).max(1)[0]
        target = rewards + self.params['algorithm']['discount_factor'] * next_q_values * (1 - dones)

        # Compute critic loss
        loss = nn.MSELoss()(q_values, target)

        # Optimize the critic network
        self.optimizers[agent].zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critics[agent].parameters(), self.params['algorithm']['max_grad_norm'])
        self.optimizers[agent].step()

    def update_target_networks(self):
        for agent in self.agents:
            self.target_critics[agent].load_state_dict(self.critics[agent].state_dict())


params = {
    "algorithm": {
        "epsilon": 0.02,
        "batch_size": 256,
        "target_critic_update": 1000,
        "discount_factor": 0.99,
        "architecture": {"hidden_size": [256, 256]},
        "max_grad_norm": 0.5,
        "n_updates": 32,
    },
    "optimizer": {
        "lr": 1e-3,
    },
}

# Initialize the PettingZoo environment
env = knights_archers_zombies_v10.env()

# Initialize the DQN agent
multi_agent_dqn = MultiAgentDQN(env, params)

# Train the agent
multi_agent_dqn.train(n_episodes=1000)

env.close()


256 256
(array([[ 0.        ,  0.3       ,  0.825     ,  0.        , -1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -1.        ],
       [ 0.02762136,  0.0390625 ,  0.        ,  0.        , -1.        ],
       [ 0.22097087,  0.3125    ,  0.        ,  0.        , -1.        ],
       [ 0.24859223,  0.3515625 ,  0.        ,  0.        , -1.        ],
       [ 0.        , -0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        , -0.     

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (256, 5) + inhomogeneous part.

In [1]:

env = knights_archers_zombies_v10.env(render_mode="human")
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    if termination or truncation:
        action = None
    else:
        
        # this is where you would insert your policy
        action = env.action_space(agent).sample()

    env.step(action)
env.close()

: 

In [ ]:
params = {
    "base_dir": "${gym_env.env_name}/dqn-S${algorithm.seed}_${current_time:}",
    "collect_stats": True,
    "save_best": False,
    "algorithm": {
        "seed": 4,
        "max_grad_norm": 0.5,
        "epsilon": 0.02,
        "n_envs": 8,
        "n_steps": 32,
        "n_updates": 32,
        "eval_interval": 2000,
        "learning_starts": 5000,
        "nb_evals": 10,
        "buffer_size": 100_000,
        "batch_size": 256,
        "target_critic_update": 1_000,
        "max_epochs": 3_000,
        "discount_factor": 0.99,
        "architecture": {"hidden_size": [256, 256]},
    },
    "gym_env": {
        "env_name": "CartPole-v1",
    },
    "optimizer": {
        "classname": "torch.optim.Adam",
        "lr": 1e-3,
    },
}

dqn = DQN(OmegaConf.create(params))
run(dqn, dqn_compute_critic_loss)
dqn.visualize_best()